In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
from torch.autograd import Variable
import pylab as plt

def est_psd(lst,d):
    S = len(lst)
    for s in range(S):
        x = lst[s]
        hatx = torch.rfft(x ,1 ,onesided=False)
        amp2 = hatx[:,0]*hatx[:,0]+hatx[:,1]*hatx[:,1]
        if s==0:
            psd = amp2/d
        else:
            psd = psd + amp2/d
    psd = psd/S
    return psd

def gauss_omega_mcmc_simulate_omega(Y_0 ,langevin_step_size ,langevin_step_num ,signal_size, beta, samplestep = 100):
    assert(signal_size %2 ==0)
    d = signal_size
    Y_tau = Y_0.clone().requires_grad_(True)
    Lambda = np.zeros(d)
    invLambda = np.zeros(d)
    for k in range(d):
        if k < d/2:
            omega = 2*math.pi * k / d
        else:
            omega = 2*math.pi * k / d - 2*math.pi
        if k==0:
            Lambda[k] = 0
            invLambda[k] = 0
        else:
            Lambda[k] = (abs(omega) ** (-beta))
            invLambda[k] = 1/Lambda[k]
    #plt.plot(Lambda)
    lst_noise = []
    lst_ytau = []
    for i in range(langevin_step_num):
        if Y_tau.grad is not None:
            Y_tau.grad.data.zero_()
        noise = torch.randn(d)
        if i % samplestep == 0:
            lst_noise.append(noise)
            lst_ytau.append(Y_tau.clone().detatch())
        Y_ftau = torch.rfft(Y_tau ,1 ,onesided=False)
        Y_aftau = Y_ftau[:,0]*Y_ftau[:,0]+Y_ftau[:,1]*Y_ftau[:,1]
        #plt.plot(Y_aftau.numpy()/d)
        beta_omega = torch.Tensor(invLambda)
        Y_mul = (Y_aftau*beta_omega)/d
        #plt.plot(Y_mul.numpy())
        log_pY = -0.5 * Y_mul.sum() # -logZ
        log_pY.backward()
        Y_diff = 0.5 * (langevin_step_size**2)*Y_tau.grad + langevin_step_size * noise
        #print(i, Y_tau.grad.norm()) # Y_tau, noise, Y_tau.grad)
        Y_tau.data.add_(Y_diff.data)
        print(i,log_pY)
    
    return lst_noise, lst_ytau, Lambda


In [2]:
langevin_step_size = 0.001
langevin_step_num = 20000
beta = 0 # 1.2 # decay of power spectrum, Lambda(om) = |om|^(-beta)
image_size = 64

Y_0 =  torch.randn(image_size)
lst_noise, lst_ytau, Lambda = gauss_omega_mcmc_simulate_omega(Y_0 ,langevin_step_size ,langevin_step_num ,image_size, beta)
psd_noise = est_psd(lst_noise,image_size)
psd_ytau = est_psd(lst_ytau,image_size)


AttributeError: 'Tensor' object has no attribute 'detatch'

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(311)
plt.plot(psd_noise.data.numpy())
plt.title('noise psd')
plt.subplot(312)
plt.plot(psd_ytau.data.numpy())
plt.title('y_tau psd')
plt.subplot(313)
plt.plot(Lambda)